In [1]:
import torch
from transformers import AutoTokenizer, EsmForMaskedLM
from transformers.models.esm.openfold_utils.protein import to_pdb, Protein as OFProtein
from transformers.models.esm.openfold_utils.feats import atom14_to_atom37
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from pyJoules.device.nvidia_device import NvidiaGPUDomain
from pyJoules.handler.pandas_handler import PandasHandler
from pyJoules.energy_meter import measure_energy

from Bio import SeqIO
import pandas as pd
import numpy as np
from time import perf_counter_ns
import matplotlib.pyplot as plt
import gc

SEQ_MAX_LEN = 8096

c:\Users\lgltan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### CUDA/Torch GPU Setup

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA GeForce RTX 4060 Ti


## Convert FASTA file to pd.DataFrame for easier processing

In [3]:
def fasta_to_dataframe(fasta_file):
    records = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):
        if len(records) >= 8000:
            break
        # Extract the ID after '>tr'
        id_info = seq_record.id.split('|')[1] # Adjust the index based on your FASTA file format
        sequence = str(seq_record.seq)
        if len(sequence) <= SEQ_MAX_LEN:
            # Generate a random index within the range of the sequence length
            random_index = np.random.randint(0, min(len(sequence), SEQ_MAX_LEN))
            # Replace the character at the random index with '<mask>'
            masked_sequence = sequence[:random_index] + '<mask>' + sequence[random_index+1:]
            
            records.append([id_info, sequence, masked_sequence])
    
    # Create a DataFrame from the records
    df = pd.DataFrame(records, columns=['ID', 'Sequence', 'Masked_Sequence'])
    return df

## Create Dataset

In [4]:
fasta_df = fasta_to_dataframe("human_protein_seq/uniprotkb_proteome_UP000005640.fasta")

In [5]:
fasta_df.head()

,ID,Sequence,Masked_Sequence
0,A0A075B6G3,MLWWEEVEDCYEREDVQKKTFTKWVNAQFSKFGKQHIENLFSDLQD...,MLWWEEVEDCYEREDVQKKTFTKWVNAQFSKFGKQHIENLFSDLQD...
1,A0A087WV00,MDAAGRGCHLLPLPAARGPARAPAAAAAAAASPPGPCSGAACAPSA...,MDAAGRGCHLLPLPAARGPARAPAAAAAAAASPPGPCSGAACAPSA...
2,A0A087WZT3,MELSAEYLREKLQRDLEAEHVLPSPGGVGQVRGETAASETQLGS,MEL<mask>AEYLREKLQRDLEAEHVLPSPGGVGQVRGETAASETQLGS
3,A0A087X1C5,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...
4,A0A087X296,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPVNPCCYYPCQHQGIC...,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPVNPCCYYPCQHQGIC...


In [6]:
fasta_df.shape

(8000, 3)

## Preparing your model and tokenizer

Now we load our model and tokenizer. If using GPU, use `model.cuda()` to transfer the model to GPU.

In [7]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t36_3B_UR50D")
model = EsmForMaskedLM.from_pretrained("facebook/esm2_t36_3B_UR50D")

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.44s/it]


In [8]:
class FastaDataset(Dataset):
    def __init__(self, fasta_df, tokenizer, max_length=512):
        self.fasta_df = fasta_df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.fasta_df)

    def __getitem__(self, idx):
        fasta_row = self.fasta_df.iloc[idx]
        latency_time_stamps = []
        # TIME LOG: start
        latency_time_stamps.append(perf_counter_ns())

        # Tokenize the input and move it to the correct device
        inputs = tokenizer(fasta_row["Masked_Sequence"], max_length=min(len(fasta_row["Masked_Sequence"]), SEQ_MAX_LEN), truncation=True, return_tensors='pt')
        
        # TIME LOG: tokenize
        latency_time_stamps.append(perf_counter_ns())
        
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # TIME LOG: move_to_gpu
        latency_time_stamps.append(perf_counter_ns())
        
        unmasked_text = fasta_row['Sequence']
        masked_text = fasta_row["Masked_Sequence"]
        mask_position = masked_text.find('<mask>')
        actual_token = unmasked_text[mask_position]
        
        with torch.no_grad():
            logits = model(**inputs).logits
        
        # TIME LOG: model
        latency_time_stamps.append(perf_counter_ns())

        # Convert the actual token to its corresponding ID
        actual_token_id = tokenizer.encode(actual_token, add_special_tokens=False)[0]

        # Prepare the target tensor
        target = torch.tensor([actual_token_id] + np.zeros(33, dtype=np.longlong), device=device).unsqueeze(0)
        
        # Calculate the cross-entropy loss
        loss = loss_fn(logits, target)
        
        return loss, latency_time_stamps, fasta_row

In [9]:
# Instantiate the dataset
fasta_dataset = FastaDataset(fasta_df, tokenizer)

# Create a DataLoader
batch_size = 8
dataloader = DataLoader(fasta_dataset, batch_size=batch_size, shuffle=False)

In [10]:
# Ensure the model is in evaluation mode
model.eval()

model.to(device)

EsmForMaskedLM(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 2560, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 2560, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-35): 36 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=2560, out_features=2560, bias=True)
              (key): Linear(in_features=2560, out_features=2560, bias=True)
              (value): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((2560,), eps=1e-05, 

# TODO: Revamp following code to implement Datasets and DataLoaders

In [11]:
print(f"Running test on {fasta_df.shape[0]} sequences.")

gc.disable()

# set up accuracy & latency info - leave this in the same code block so that it resets each time
accuracy = []
latency = pd.DataFrame(columns=['start', 'tokenize', 'move_to_gpu', 'model', "end"])
sequence_length = []
# memory_usage = []
ctr = 0

loss_fn = nn.CrossEntropyLoss()
pandas_handler = PandasHandler()

@measure_energy(domains=[NvidiaGPUDomain(0)], handler=pandas_handler)
def run(accuracy, latency, dataloader):
    loss, latency_time_stamps, fasta_row = next(iter(dataloader))

    # TIME LOG: end
    latency_time_stamps.append(perf_counter_ns())

    # Append the loss to a list or use it as needed
    accuracy.append(loss.item())
        
    # memory_usage.append(torch.cuda.memory_allocated())

    latency.loc[len(latency.index)] = latency_time_stamps
    
    return len(fasta_row["Sequence"])

torch.cuda.empty_cache()

for ctr in range(fasta_df.shape[0]):
    if ctr % 100 == 0:
        percentage = "{:.2f}".format(ctr * 100 / fasta_df.shape[0])
        print(f"Processing... {percentage}%")

    torch.cuda.empty_cache()
    # try to save as much memory as possible by reducing fragmentation
    gc.collect()
    
    # load model to GPU
    model.to(device)

    sequence_length.append(run(accuracy, latency, dataloader))
    
    # try to remove anything no longer needed on the GPU mem
    gc.collect()
            
gc.enable()
gpu_util_df = pandas_handler.get_dataframe()

Running test on 8000 sequences.
Processing... 0.00%


RuntimeError: stack expects each tensor to be equal size, but got [1, 3687, 33] at entry 0 and [1, 1059, 33] at entry 1

Due to memory constraints, sequences over 10k characters in length have been dropped from the dataset for this model; hence only 1 set of graphs are made

### Cross Entropy Loss x Sequence Length

In [ ]:
# Create a scatter plot
plt.scatter(sequence_length, accuracy)

# Label the axes
plt.xlabel('Sequence Length')
plt.ylabel('Cross Entropy Loss')

# Add a title
plt.title('Sequence Length vs. Cross Entropy Loss')

# Display the plot
plt.show()

## Latency/Time Log

In [ ]:
# latency = pd.DataFrame(columns=['start', 'tokenize', 'move_to_gpu', 'model', "end"])
# get_cel = get cross entropy loss
latency_df = pd.DataFrame(columns=['tokenization', 'move_inputs_to_gpu', 'model_process', 'cross_entropy_loss'])

latency_df["tokenization"] = latency["tokenize"] - latency["start"]
latency_df["move_inputs_to_gpu"] = latency["move_to_gpu"] - latency["tokenize"]
latency_df["model_process"] = latency["model"] - latency["move_to_gpu"]
latency_df["cross_entropy_loss"] = latency["end"] - latency["model"]

latency_sums = latency_df.sum()
# Create a bar plot
latency_sums.plot(kind='bar')

# Add title and labels
plt.title('Total Latency per Step')

# Display the plot
plt.show()

### Latency & GPU Usage vs Sequence Length

In [ ]:
gpu_util_df.head()

In [ ]:
# Normalize Duration
duration = gpu_util_df["duration"] * 100000

lat_gpu_slen = {"Latency": duration, "GPU_Utilization": gpu_util_df["nvidia_gpu_0"], "Sequence_Length": sequence_length}
lat_gpu_slen_df = pd.DataFrame(lat_gpu_slen)
lat_gpu_slen_df = lat_gpu_slen_df.sort_values(by = 'Sequence_Length')

# Plot multiple lines
plt.plot(lat_gpu_slen_df["Sequence_Length"], lat_gpu_slen_df["GPU_Utilization"], label='GPU_Utilization')
plt.plot(lat_gpu_slen_df["Sequence_Length"], lat_gpu_slen_df["Latency"], label='Latency')

plt.xlabel('Sequence Length')
plt.ylabel('10ns / Joule')

plt.title("Latency/GPU Util vs Sequence Length")
plt.legend()

plt.show()